In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.text import OffsetFrom
from matplotlib import ticker, dates
#import imgkit
%matplotlib inline
pd.set_option('max_rows',200)
pd.set_option('min_rows',100)
from ipywidgets import interact, interactive,fixed, interact_manual
import ipywidgets as widgets

## Import all Tables

### Covid Raw data

In [2]:
url="http://187.191.75.115/gobmx/salud/datos_abiertos/datos_abiertos_covid19.zip"
covid = pd.read_csv(url,parse_dates=True, encoding="mac_roman",dtype={"ENTIDAD_RES":'object',"MUNICIPIO_RES":'object'})
covid["FECHA_ACTUALIZACION"]=pd.to_datetime(covid["FECHA_ACTUALIZACION"])#,format="%d/%m/%Y")
covid["FECHA_INGRESO"]=pd.to_datetime(covid["FECHA_INGRESO"])#,format="%d/%m/%Y")
covid["FECHA_SINTOMAS"]=pd.to_datetime(covid["FECHA_SINTOMAS"])#,format="%d/%m/%Y")

URLError: <urlopen error [Errno 60] Operation timed out>

In [ ]:
fecha_actualizaion=covid.iloc[-1,:]["FECHA_ACTUALIZACION"]

### Fecha de Defunsion

In [ ]:
mask=covid["FECHA_DEF"]=="9999-99-99"
covid.loc[mask,"FECHA_DEF"]="NaN"
covid["FECHA_DEF"]=pd.to_datetime(covid["FECHA_DEF"])#,format="%d/%m/%Y")

### Catalogs

In [ ]:
origen=pd.read_excel("Catalogos_0412.xlsx","Catálogo ORIGEN",index_col="CLAVE")

In [ ]:
sector=pd.read_excel("Catalogos_0412.xlsx","Catálogo SECTOR",index_col="CLAVE")

In [ ]:
sexo=pd.read_excel("Catalogos_0412.xlsx","Catálogo SEXO",index_col="CLAVE")

In [ ]:
entidad=pd.read_excel("Catalogos_0412.xlsx","Catálogo ENTIDADES",dtype='object')

In [ ]:
entidad.set_index("CLAVE_ENTIDAD",inplace=True, drop=False)

In [ ]:
paciente=pd.read_excel("Catalogos_0412.xlsx","Catálogo PACIENTE",index_col="CLAVE")

In [ ]:
boo=pd.read_excel("Catalogos_0412.xlsx","Catálogo BOOL",inde_col="CLAVE")

In [ ]:
nacionalidad=pd.read_excel("Catalogos_0412.xlsx","Catálogo NACIONALIDAD",index_col="CLAVE")

In [ ]:
resultado=pd.read_excel("Catalogos_0412.xlsx","Catálogo RESULTADO",index_col="CLAVE")

In [ ]:
municipio=pd.read_excel("Catalogos_0412.xlsx","Catálogo MUNICIPIOS",dtype='object')

In [ ]:
municipio=municipio.merge(entidad[["ENTIDAD_FEDERATIVA","ABREVIATURA"]], "left", left_on="CLAVE_ENTIDAD", right_index=True)

### Fixing Catalogo Municipio

In [ ]:
covid["ENTIDAD_RES"]=covid["ENTIDAD_RES"].apply(str)
covid["MUNICIPIO_RES"]=covid["MUNICIPIO_RES"].apply(str)
mun=covid["ENTIDAD_RES"]+covid["MUNICIPIO_RES"]
covid.insert(9,"MUNICIPIOS",mun)
#covid["ENTIDAD_RES"]=covid["ENTIDAD_RES"].astype(int)
#covid["MUNICIPIOS"]=covid["MUNICIPIOS"].astype(int)

municipio["CLAVE_MUNICIPIO"]=municipio["CLAVE_MUNICIPIO"].apply(str)
municipio["CLAVE_ENTIDAD"]=municipio["CLAVE_ENTIDAD"].apply(str)
municipio["ENT&MUN"]=municipio["CLAVE_ENTIDAD"]+municipio["CLAVE_MUNICIPIO"]
#municipio["ENT&MUN"]=municipio["ENT&MUN"].astype(int)
municipio.drop_duplicates(subset="ENT&MUN",inplace=True)
municipio.drop(["CLAVE_ENTIDAD","CLAVE_MUNICIPIO"],axis=1,inplace=True)
municipio.set_index("ENT&MUN",inplace=True,drop=False)

### Merging Catalogs in covid DataFrame

In [ ]:
covid=covid.merge(entidad, "left", left_on="ENTIDAD_RES", right_index=True)
covid=covid.drop(["ENTIDAD_UM","ENTIDAD_NAC","ENTIDAD_RES","CLAVE_ENTIDAD"],axis=1)

In [ ]:
covid=covid.merge(municipio["MUNICIPIO"], "left", left_on="MUNICIPIOS", right_index=True)
covid=covid.drop(["MUNICIPIOS","MUNICIPIO_RES"],axis=1)

In [ ]:
covid=covid.merge(paciente,'left',left_on="TIPO_PACIENTE",right_index=True)
covid=covid.drop("TIPO_PACIENTE",axis=1)

In [ ]:
covid=covid.merge(sector,'left',left_on="SECTOR",right_index=True)
covid=covid.drop("SECTOR",axis=1)

In [ ]:
covid=covid.merge(sexo,'left',left_on="SEXO",right_index=True)
covid=covid.drop("SEXO",axis=1)

In [ ]:
covid=covid.merge(resultado,'left',left_on="RESULTADO",right_index=True)
covid=covid.drop("RESULTADO",axis=1)

In [ ]:
covid=covid.drop(['NACIONALIDAD','HABLA_LENGUA_INDIG','MIGRANTE', 
            'PAIS_NACIONALIDAD','PAIS_ORIGEN'],axis=1)

### Converting Si-No to Boolean

In [ ]:
to_boolean=["INTUBADO","NEUMONIA",'EMBARAZO', 'DIABETES',
       'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO','UCI']

def convert_sino(value):
    if value == 1:
        return True
    elif value == 2:
        return False
    else:
        return ""

for i in to_boolean:
    covid[i]=covid[i].apply(convert_sino).astype(bool)

### Add other Columns

In [ ]:
covid["SOSPECHOSO"]=True
covid["CONFIRMADO"] = covid["RESULTADOS"]=="Positivo SARS-CoV-2"
covid["NEGATIVO"]=covid["RESULTADOS"]=="No positivo SARS-CoV-2"
covid["DEFUNSION"]=~covid["FECHA_DEF"].isna()
covid["MUERTE x COVID"]=covid["DEFUNSION"]&covid["CONFIRMADO"]

mask1=covid["CONFIRMADO"]&~covid["MUERTE x COVID"]
mask2=(covid["FECHA_ACTUALIZACION"]-covid["FECHA_INGRESO"])>pd.Timedelta(14, unit="D")
covid["RECUPERADO"]=mask1&mask2

mask1=covid["CONFIRMADO"]
mask3=~covid["RECUPERADO"]
mask4=~covid["MUERTE x COVID"]
covid["ACTIVO"]=mask1&mask3&mask4

In [ ]:
covid.rename(columns={"ENTIDAD_FEDERATIVA":"ESTADO"},inplace=True)

In [ ]:
padecimientos=["DIABETES","EPOC","ASMA","INMUSUPR","HIPERTENSION",
            "CARDIOVASCULAR","OBESIDAD","RENAL_CRONICA","TABAQUISMO"]

In [ ]:
covid["PADECIMIENTO"]=covid[padecimientos].sum(axis=1)!=0
covid["NO PADECIMIENTO"]=covid[padecimientos].sum(axis=1)==0

In [ ]:
covid.loc[covid['RECUPERADO'] ==True, "STATUS"] = "RECUPERADO"
covid.loc[covid['ACTIVO'] ==True, "STATUS"] = "ACTIVO"
covid.loc[covid['MUERTE x COVID'] ==True, "STATUS"] = "DECESO"
covid["STATUS"]=covid["STATUS"].astype("category")

In [ ]:
covid.iloc[:,22:29]=covid.iloc[:,22:29].astype("category")

## Pivots & Graphs

In [ ]:
import seaborn as sns
sns.set(context="notebook",style="white")
plt.rcParams['figure.figsize'] = (14, 5)
plt.rcParams['figure.dpi'] = 75
coma=mpl.ticker.StrMethodFormatter('{x:,g}' )
percen=mpl.ticker.PercentFormatter()
formatter = mpl.dates.DateFormatter("%d/%m")

### Resumen nivel Nacional

In [ ]:
pais=covid.pivot_table(values=["SOSPECHOSO","CONFIRMADO","MUERTE x COVID","RECUPERADO"],
                             aggfunc=["sum","mean","count"],index="FECHA_INGRESO")
pais["mean"]=pais["sum"].rolling(21).mean()
pais["count"]=pais["sum"].cumsum()
pais.rename(columns={"sum":"Nuevos Casos por día","mean":"21 day moving avg",
                     "count": "Acumulado"},inplace=True)

### Resumen por Estado

In [ ]:
porestado=covid.pivot_table(values=["SOSPECHOSO","CONFIRMADO","MUERTE x COVID","RECUPERADO"],
                             aggfunc=["sum","mean","count"],index="FECHA_INGRESO",
                             columns="ABREVIATURA")
porestado["mean"]=porestado["sum"].rolling(21).mean()
porestado["count"]=porestado["sum"].cumsum()
porestado.rename(columns={"sum":"Nuevos Casos por día","mean":"21 day moving avg",
                         "count": "Acumulado"},inplace=True)

In [ ]:
def indice_mortalidad(estado):
    
    mortandad=pais[("Acumulado","MUERTE x COVID")]/pais[("Acumulado","CONFIRMADO")]
    mortandad=mortandad.to_frame()
    mortandad["Promedio Móvil"]=mortandad[0].rolling(21).mean()
    mortandad.dropna(axis=0,inplace=True)
    mortandad.columns=["Mortandad",'Promedio Móvil']
    mortandad=mortandad.loc["2020-03-20":pd.Timestamp.today()-pd.Timedelta(5,unit="D")]
    
    mask1=entidad["ENTIDAD_FEDERATIVA"]==estado
    edo=entidad[mask1].iat[0,2]
    mortandad2=porestado.loc["2020-03-20":pd.Timestamp.today()-pd.Timedelta(5,unit="D"),("Acumulado",("CONFIRMADO","MUERTE x COVID"),edo)]

    mortandad2.columns=mortandad2.columns.to_flat_index()
    mortandad2.columns=["CONFIRMADO","MUERTE x COVID"]

    mortandad2["Mortandad"]=mortandad2["MUERTE x COVID"]/mortandad2["CONFIRMADO"]
    mortandad2["Promedio Móvil"]=mortandad2["Mortandad"].rolling(21).mean()

    fig, (ax,ax2)=plt.subplots(2,1, constrained_layout=True)
    ax.vlines(mortandad.index,ymin=0, ymax=mortandad["Mortandad"]*100,color="gray")
    ax.plot(mortandad["Promedio Móvil"]*100,color="r",label="Promedio movil 21 días")
    ax.axhline(y=9)
    ax.xaxis.set_major_locator(ticker.MaxNLocator(8))
    ax.xaxis.set_major_locator(ticker.NullLocator())
    ax.yaxis.set_major_formatter(percen)
    bbox_args = dict(boxstyle="round", fc="1")
    arrow_args = dict(arrowstyle="->",color="r")
    ax.annotate('Promedio Mundial',
                    xy=(0.5, 9), xycoords=('axes fraction','data'),
                    xytext=(-30, -50), textcoords='offset points',
                    ha="center", va="top",
                    bbox=bbox_args,
                    arrowprops=arrow_args)
    ax.set_ylabel('Indice de Mortalidad')
    ax.set_title("Evolución del Indice de Mortalidad por COVID - Nivel Nacional")
    ax.legend()

    ax2.vlines(mortandad2.index,ymin=0, ymax=mortandad2["Mortandad"]*100,color="gray")
    ax2.plot(mortandad2["Promedio Móvil"]*100,color="r")
    ax2.axhline(y=9)
    ax2.xaxis.set_major_locator(ticker.MaxNLocator(8))
    ax2.xaxis.set_major_formatter(formatter)
    ax2.yaxis.set_major_formatter(percen)
    ax2.annotate('Promedio Mundial',
                    xy=(0.5, 9), xycoords=('axes fraction','data'),
                    xytext=(30, 50), textcoords='offset points',
                    ha="center", va="top",
                    bbox=bbox_args,
                    arrowprops=arrow_args)
    ax2.set_ylabel('Indice de Mortalidad')
    ax2.set_title("Evolución del Indice de Mortalidad por COVID  --  " +str(estado))
    fig.savefig("mortandad");

In [ ]:
def situacion_nacional(item):
    
    mortandad=pais[("Acumulado","MUERTE x COVID")]/pais[("Acumulado","CONFIRMADO")]
    mortandad=mortandad.to_frame()
    mortandad["Promedio Móvil"]=mortandad[0].rolling(21).mean()
    mortandad.dropna(axis=0,inplace=True)
    mortandad.columns=["Mortandad",'Promedio Móvil']
    mortandad=mortandad.loc["2020-03-20":pd.Timestamp.today()-pd.Timedelta(5,unit="D")]

    i=item
    if item =="CONFIRMADO":
        x=5
    elif item=="SOSPECHOSO":
        x=1
    elif item=="MUERTE x COVID":
        x=14
    
    idx=pd.IndexSlice
    graph2=pais.loc["2020-03-20":pd.Timestamp.today()-pd.Timedelta(x,unit="D")
               ,idx[("Nuevos Casos por día","21 day moving avg"),(item)]]
    
    fig, (ax,ax2)=plt.subplots(1,2, constrained_layout=True)
    
    ax.plot(graph2["21 day moving avg"], color="g", 
             label="Promedio móvil 21 días")
    ax.vlines(graph2.index,ymin=0,ymax=graph2["Nuevos Casos por día"],
              label="Nuevos Casos por día",color="gray")
   
    ax.legend(loc="upper left")
    ax.set_ylabel('Número de Casos')
    ax.xaxis.set_major_locator(ticker.MaxNLocator(8))
    ax.xaxis.set_major_formatter(formatter)
    ax.yaxis.set_major_formatter(coma)
    ax.set_title("Desarrollo Casos COVID  "+str(i)+"- Nivel Nacional")     
    
    
    ax2.vlines(mortandad.index,ymin=0, ymax=mortandad["Mortandad"]*100,color="gray")
    ax2.plot(mortandad["Promedio Móvil"]*100,color="r",label="Promedio movil 21 días")
    ax2.axhline(y=9)
    ax2.xaxis.set_major_locator(ticker.MaxNLocator(8))
    ax2.xaxis.set_major_formatter(formatter)
    ax2.yaxis.set_major_formatter(percen)
    bbox_args = dict(boxstyle="round", fc="1")
    arrow_args = dict(arrowstyle="->",color="r")
    ax2.annotate('Promedio Mundial',
                    xy=(0.5, 9), xycoords=('axes fraction','data'),
                    xytext=(-30, -50), textcoords='offset points',
                    ha="center", va="top",
                    bbox=bbox_args,
                    arrowprops=arrow_args)
    ax2.set_ylabel('Indice de Mortalidad')
    ax2.set_title("Evolución del Indice de Mortalidad por COVID - Nivel Nacional")
    ax2.legend()

   
    fig.savefig("nacional");

In [ ]:
def situacion_estatal(item, estado):
    
    mask1=entidad["ENTIDAD_FEDERATIVA"]==estado
    edo=entidad[mask1].iat[0,2]
    
    mortandad2=porestado.loc["2020-03-20":pd.Timestamp.today()-pd.Timedelta(5,unit="D"),("Acumulado",("CONFIRMADO","MUERTE x COVID"),edo)]
    mortandad2.columns=mortandad2.columns.to_flat_index()
    mortandad2.columns=["CONFIRMADO","MUERTE x COVID"]
    mortandad2["Mortandad"]=mortandad2["MUERTE x COVID"]/mortandad2["CONFIRMADO"]
    mortandad2["Promedio Móvil"]=mortandad2["Mortandad"].rolling(21).mean()
    
    i=item
    if item =="CONFIRMADO":
        x=5
    elif item=="SOSPECHOSO":
        x=1
    elif item=="MUERTE x COVID":
        x=14
    
    idx=pd.IndexSlice
    graph=porestado.loc["2020-03-20":pd.Timestamp.today()-pd.Timedelta(x,unit="D")
               ,idx[("Nuevos Casos por día","21 day moving avg"),(item),edo]]
    
    fig, (ax,ax2)=plt.subplots(1,2, constrained_layout=True)
    
    ax.vlines(graph.index, ymin=0,ymax=graph["Nuevos Casos por día"],
               label="Nuevos Casos por día",color="gray")
    ax.plot(graph["21 day moving avg"], color="g",
            label="Promedio móvil 21 días")
    ax.legend(loc="upper left")
    ax.set_xlabel('Fecha de Ingreso')
    ax.set_ylabel('Número de Casos')
    
    ax.set_title("Desarrollo Casos COVID  "+str(i)+" "+str(estado))
    ax.xaxis.set_major_locator(ticker.MaxNLocator(8))
    ax.xaxis.set_major_formatter(formatter)
    
    bbox_args = dict(boxstyle="round", fc="1")
    arrow_args = dict(arrowstyle="->",color="r")
    ax2.vlines(mortandad2.index,ymin=0, ymax=mortandad2["Mortandad"]*100,color="gray")
    ax2.plot(mortandad2["Promedio Móvil"]*100,color="r")
    ax2.axhline(y=9)
    ax2.xaxis.set_major_locator(ticker.MaxNLocator(8))
    ax2.xaxis.set_major_formatter(formatter)
    ax2.yaxis.set_major_formatter(percen)
    ax2.annotate('Promedio Mundial',
                    xy=(0.5, 9), xycoords=('axes fraction','data'),
                    xytext=(30, 50), textcoords='offset points',
                    ha="center", va="top",
                    bbox=bbox_args,
                    arrowprops=arrow_args)
    ax2.set_xlabel('Fecha de Ingreso')
    ax2.set_ylabel('Indice de Mortalidad')
    ax2.set_title("Evolución del Indice de Mortalidad por COVID  --  " +str(estado))
    fig.savefig("estatal");

In [ ]:


def evolucion_covid(item,estado):
    i=item
    if item =="CONFIRMADO":
        x=5
    elif item=="SOSPECHOSO":
        x=1
    elif item=="MUERTE x COVID":
        x=14
    mask1=entidad["ENTIDAD_FEDERATIVA"]==estado
    edo=entidad[mask1].iat[0,2]
    
    idx=pd.IndexSlice
    graph=porestado.loc["2020-03-20":pd.Timestamp.today()-pd.Timedelta(x,unit="D")
               ,idx[("Nuevos Casos por día","21 day moving avg"),(item),edo]]
    graph2=pais.loc["2020-03-20":pd.Timestamp.today()-pd.Timedelta(x,unit="D")
               ,idx[("Nuevos Casos por día","21 day moving avg"),(item)]]
    
    
    fig, (ax1,ax2)=plt.subplots(2,1,constrained_layout=True)
    ax1.plot(graph2["21 day moving avg"], color="g", 
             label="Promedio móvil 21 días")
    ax1.vlines(graph2.index,ymin=0,ymax=graph2["Nuevos Casos por día"],
              label="Nuevos Casos por día",color="gray")
   
    ax1.legend(loc="upper left")
    ax1.set_ylabel('Número de Casos')
    ax1.xaxis.set_major_locator(ticker.NullLocator())
    ax1.yaxis.set_major_formatter(coma)
    ax1.set_title("Desarrollo Casos COVID  "+str(i)+" a nivel Nacional")
    
    
    ax2.vlines(graph.index, ymin=0,ymax=graph["Nuevos Casos por día"],
              label="Nuevos Casos por día",color="gray")
    ax2.plot(graph["21 day moving avg"], color="g",
            label="Promedio móvil 21 días")
    ax2.legend(loc="upper left")
    ax2.set_xlabel('Fecha de Ingreso')
    ax2.set_ylabel('Número de Casos')
    
    ax2.set_title("Desarrollo Casos COVID  "+str(i)+" "+str(estado))
    ax2.xaxis.set_major_locator(ticker.MaxNLocator(8))
    ax2.xaxis.set_major_formatter(formatter)
    fig.savefig("evolucion.png");


In [ ]:
def por_edad(estado):
    
    mask0=entidad["ENTIDAD_FEDERATIVA"]==estado
    edo=entidad[mask0].iat[0,2]
    
    mask1=~covid["FECHA_DEF"].isna()
    mask2=covid["RESULTADOS"]=="Positivo SARS-CoV-2"
    mask3=covid["ABREVIATURA"]==edo
    data=covid[mask1&mask2&mask3][["EDAD","FECHA_DEF","H/M"]]
    xx=data["H/M"]=="MUJER"
    xy=data["H/M"]=='HOMBRE'
    mu=data["EDAD"].mean()
    total=data["EDAD"].count()
    sigma=data["EDAD"].std()
    fecha="{:%d/%m/%y}".format(pd.Timestamp.today())
    text="{} fallecimientos al día {}".format(total,fecha)

    nbins=20
    g=sns.FacetGrid(data, col="H/M", height=7)
    g.map(plt.hist,"EDAD")
    plt.subplots_adjust(top=0.8)
    g.fig.suptitle('Fallecimientos por EDAD y SEXO - {}'.format(estado))
    g.savefig("por_edad");

In [ ]:
def padecimientos(estado):
    
    mask0=entidad["ENTIDAD_FEDERATIVA"]==estado
    edo=entidad[mask0].iat[0,2]
    
    mask=covid["ABREVIATURA"]==edo

    mask1=covid["RECUPERADO"]
    recuperado=covid[mask1&mask]

    mask2=covid["MUERTE x COVID"]
    muerte=covid[mask2&mask]

    mask3=covid["NEGATIVO"]
    negativo=covid[mask3&mask]

    mask4=covid["CONFIRMADO"]
    positivo=covid[mask4&mask]

    mask5=covid["SOSPECHOSO"]
    sospechoso=covid[mask5&mask]

    padec=["PADECIMIENTO","DIABETES","EPOC","ASMA","INMUSUPR","HIPERTENSION",
       "CARDIOVASCULAR","OBESIDAD","RENAL_CRONICA","TABAQUISMO","NO PADECIMIENTO"]

    condiciones=pd.DataFrame()
    condiciones["SOSPECHOSOS"]=sospechoso[padec].mean()
    condiciones["NEGATIVO"]=negativo[padec].mean()
    condiciones["POSITIVO"]=positivo[padec].mean()
    condiciones["RECUPERADOS"]=recuperado[padec].mean()
    condiciones["MUERTE"]=muerte[padec].mean()

    cm = sns.light_palette("orange", as_cmap=True)
    s = condiciones.style
    s.background_gradient(cmap=cm)
    s.format("{:.2%}")

    return s

In [ ]:
def hospitalizacion(estado):
        
    mask1=entidad["ENTIDAD_FEDERATIVA"]==estado
    edo=entidad[mask1].iat[0,2]
    
    mask1=covid["ABREVIATURA"]==edo
    mask2=covid["PACIENTE"]=="HOSPITALIZADO"
    mask3=covid["UCI"]
    status=covid[mask1 & mask2]
    uci=covid[mask1 & mask3]
    g=sns.catplot(y="INSTITUCION",data=status,col="UCI",hue="STATUS",
                  height=7,kind='count',sharex=False,legend_out=False,)
    g.set(xscale="log")
    plt.subplots_adjust(top=0.8)
    g.fig.suptitle('Pacientes Hospitalizados - {}'.format(estado))
    g.savefig("hosp");

In [ ]:
def resumen_por_estado():
    new=covid.pivot_table(values=["CONFIRMADO"],
                                 aggfunc=["sum"],index="FECHA_INGRESO",
                                 columns="ABREVIATURA")
    new.columns=entidad["ABREVIATURA"][0:-1]
    new=new.loc["2020-01-01":pd.Timestamp.today()-pd.Timedelta(5,unit="D")]

    cum=new.cumsum()

    rolling=new.rolling(21).mean()

    new_death=covid.pivot_table(values=["MUERTE x COVID"],
                                 aggfunc=["sum"],index="FECHA_INGRESO",
                                 columns="ABREVIATURA")
    new_death.columns=entidad["ABREVIATURA"][0:-1]
    new_death=new_death.loc["2020-01-01":pd.Timestamp.today()-pd.Timedelta(5,unit="D")]

    cum_death=new_death.cumsum()
    roll_death=new_death.rolling(21).mean()
    mortality=cum_death/cum

    resumen=pd.DataFrame()

    poblacion=pd.read_csv("Tabulado.csv",encoding="utf_8")["Total.2"]
    poblacion.index=entidad.iloc[0:-1,2]
    resumen["POBLACION"]=poblacion

    resumen["NUEVOS"]=new.iloc[-1,:].astype(int)
    resumen["PROMEDIO NUEVOS CASOS"]=rolling.iloc[-1,:].round(2)
    resumen["CAMBIO EN PROMEDIO DIARIO"]=(rolling.iloc[-1,:]-rolling.iloc[-5,:])/rolling.iloc[-5,:].round(2)
    resumen["CASOS TOTALES"]=cum.iloc[-1,:].astype(int)


    resumen["ULTIMOS DECESOS"]=new_death.iloc[-1,:]
    resumen["PROMEDIO MUERTES"]=roll_death.iloc[-1,:].round(2)
    resumen["CAMBIO EN PROMEDIO DE DECESOS"]=(roll_death.iloc[-1,:]-roll_death.iloc[-5,:])/roll_death.iloc[-5,:].round(2)
    resumen["DECESOS TOTALES"]=cum_death.iloc[-1,:]
    resumen["DECESOS x MILLON"]=(resumen["DECESOS TOTALES"]/resumen["POBLACION"]*1000000).round(2)
    resumen["MORTALITY"]=mortality.iloc[-1,:].round(2)
    ind=entidad.iloc[0:-1,1]
    resumen.index=ind
    resumen=resumen.drop("POBLACION",axis=1)

    resumen=resumen.style.format({'MORTALITY':"{:.2%}","CAMBIO EN PROMEDIO DIARIO":"{:.2%}"
                             ,"CAMBIO EN PROMEDIO DE DECESOS":"{:.2%}","NUEVOS":'{:,}',"CASOS TOTALES":'{:,}',
                           "PROMEDIO NUEVOS CASOS":'{:,}',"ULTIMOS DECESOS":'{:,}',"PROMEDIO MUERTES":'{:,}',
                           "DECESOS TOTALES":'{:,}',"POBLACION":'{:,}',"DECESOS x MILLON":"{:,}"})

    from matplotlib import cm
    cm1=sns.diverging_palette(220, 22, sep=20, as_cmap=True)
    cm=sns.diverging_palette(220, 20, sep=20, as_cmap=True)
    #cm = sns.dark_palette("blue", as_cmap=True)
    resumen.background_gradient(cmap=cm1,axis=0,low=-0.01,high=0.3, 
                            subset=["MORTALITY","CAMBIO EN PROMEDIO DE DECESOS",
                                    "CAMBIO EN PROMEDIO DIARIO"])
    resumen.background_gradient(cmap=cm,axis=0,low=-0.3,high=0.3, 
                            subset=["DECESOS x MILLON","CASOS TOTALES"])
    return resumen

In [ ]:
def resumen_por_municipio(estado):

    mask0=entidad["ENTIDAD_FEDERATIVA"]==estado
    edo=entidad[mask0].iat[0,2]
    
    mask=covid["ABREVIATURA"]==edo
    mask2=municipio["ABREVIATURA"]==edo
    lista_munic=municipio["MUNICIPIO"][mask2].sort_values()
    mask3=covid["MUNICIPIO"].isin(lista_munic)
    munc=covid[mask3&mask]

    new=munc.pivot_table(values=["CONFIRMADO"],
                                 aggfunc=["sum"],index="FECHA_INGRESO",
                                 columns="MUNICIPIO")

    new.columns=new.columns.get_level_values(2)
    new=new.loc["2020-01-01":pd.Timestamp.today()-pd.Timedelta(5,unit="D")]
    new=new.fillna(0)

    cum=new.cumsum(skipna=True)
    rolling=new.rolling(21).mean().round(2)

    new_death=munc.pivot_table(values=["MUERTE x COVID"],
                                 aggfunc=["sum"],index="FECHA_INGRESO",
                                 columns="MUNICIPIO")
    new_death.columns=new_death.columns.get_level_values(2)
    new_death=new_death.loc["2020-01-01":pd.Timestamp.today()-pd.Timedelta(5,unit="D")]
    new_death=new_death.fillna(0)

    cum_death=new_death.cumsum()
    roll_death=new_death.rolling(21).mean().round(2)


    resumen=municipio["MUNICIPIO"][mask2].sort_values().to_frame().set_index("MUNICIPIO")
    resumen=resumen.drop("NO ESPECIFICADO")

    NUEVOS_CASOS=new.iloc[-1,:]
    NUEVOS_CASOS.name="NUEVOS CASOS"
    resumen=resumen.merge(NUEVOS_CASOS, left_index=True, right_index=True)

    PROMEDIO_NUEVOS_CASOS=rolling.iloc[-1,:].round(2)
    PROMEDIO_NUEVOS_CASOS.name="PROMEDIO NUEVOS CASOS"
    resumen=resumen.merge(PROMEDIO_NUEVOS_CASOS, left_index=True, right_index=True)

    CAMBIO_EN_PROMEDIO_DIARIO=(rolling.iloc[-1,:]-rolling.iloc[-5,:])/rolling.iloc[-5,:].round(2)
    CAMBIO_EN_PROMEDIO_DIARIO.name="CAMBIO EN PROMEDIO DIARIO"
    resumen=resumen.merge(CAMBIO_EN_PROMEDIO_DIARIO, left_index=True, right_index=True)

    CASOS_TOTALES=cum.iloc[-1,:]
    CASOS_TOTALES.name="CASOS TOTALES"
    resumen=resumen.merge(CASOS_TOTALES, left_index=True, right_index=True)

    ULTIMOS_DECESOS=new_death.iloc[-1,:]
    ULTIMOS_DECESOS.name="ULTIMOS DECESOS"
    resumen=resumen.merge(ULTIMOS_DECESOS, left_index=True, right_index=True)

    PROMEDIO_MUERTES=roll_death.iloc[-1,:].round(2)
    PROMEDIO_MUERTES.name="PROMEDIO MUERTES"
    resumen=resumen.merge(PROMEDIO_MUERTES, left_index=True, right_index=True)

    CAMBIO_EN_PROMEDIO_DE_DECESOS=(roll_death.iloc[-1,:]-roll_death.iloc[-5,:])/roll_death.iloc[-5,:].round(2)
    CAMBIO_EN_PROMEDIO_DE_DECESOS.name="CAMBIO EN PROMEDIO DE DECESOS"
    resumen=resumen.merge(CAMBIO_EN_PROMEDIO_DE_DECESOS, left_index=True, right_index=True)

    DECESOS_TOTALES=cum_death.iloc[-1,:]
    DECESOS_TOTALES.name="DECESOS TOTALES"
    resumen=resumen.merge(DECESOS_TOTALES, left_index=True, right_index=True)


    resumen["MORTALITY"]=resumen["DECESOS TOTALES"]/resumen["CASOS TOTALES"]
    resumen=resumen.replace([np.inf,-np.inf],0)

    resumen=resumen.style.format({'MORTALITY':"{:.2%}","CAMBIO EN PROMEDIO DIARIO":"{:.2%}"
                             ,"CAMBIO EN PROMEDIO DE DECESOS":"{:.2%}","NUEVOS CASOS":'{:,}',"CASOS TOTALES":'{:,}',
                           "PROMEDIO NUEVOS CASOS":'{:,}',"ULTIMOS DECESOS":'{:,}',"PROMEDIO MUERTES":'{:,}',
                           "DECESOS TOTALES":'{:,}',"POBLACION":'{:,}'},na_rep="-")

    from matplotlib import cm
    cm1=sns.diverging_palette(220, 22, sep=20, as_cmap=True)
    cm=sns.diverging_palette(220, 20, sep=20, as_cmap=True)
    resumen.background_gradient(cmap=cm1,axis=0,low=-0.1,high=0.3, 
                            subset=["MORTALITY","CAMBIO EN PROMEDIO DE DECESOS",
                                    "CAMBIO EN PROMEDIO DIARIO"])
    resumen.background_gradient(cmap=cm,axis=0,low=-0.3,high=0.3, 
                            subset=["CASOS TOTALES", "NUEVOS CASOS","DECESOS TOTALES"])
        
    return resumen

In [ ]:
def present_analysis(item,edo):

    mask1=entidad["ABREVIATURA"]==edo
    edo_name=entidad[mask1].iat[0,0]
    
    indice_mortalidad(edo)
    evolucion_covid(item,edo)
    por_edad(edo)
    hospitalizacion(edo)
    
    estados=resumen_por_estado()
    municipios=resumen_por_municipio(edo)
    padec=padecimientos(edo)
    
    with pd.ExcelWriter('COVID-MX.xlsx') as writer:  
        estados.to_excel(writer, sheet_name='ESTADOS')
        municipios.to_excel(writer, sheet_name="Municipios - {}".format(edo_name))
        padec.to_excel(writer, sheet_name="Padecimientos - {}".format(edo_name))
    

In [ ]:
#present_analysis(1,"MS")

# Reporte sobre el Desarrollo de Covid en Mexico.
Fuente de Datos: Secretaria de Salud:http://187.191.75.115/gobmx/salud/datos_abiertos/

Dado el retrazo en actualizar los datos, las graficas muestran datos con atrazo de 5 - 14 dias son el fin de mostrar una mejor aproximación a la situación real. 

## Situacion de la Evolucion de COVID-19 a Nivel Nacional

In [ ]:
estados=entidad.iloc[:,1]
items=["CONFIRMADO","SOSPECHOSO","MUERTE x COVID"]
interact(situacion_nacional, item=items);

### Situación por Estado

In [ ]:
resumen_por_estado()

## Situacion Estatal

In [ ]:
interact(situacion_estatal,estado=estados,item=items);

### Situación por Municipio

In [ ]:
interact(resumen_por_municipio, estado=estados);

### Estado de los Pacientes Hospitalizados

In [ ]:
interact(hospitalizacion, estado=estados);

### Distribución de los Fallecimientos

In [ ]:
interact(por_edad, estado=estados);

### Padecimientos

In [ ]:
interact(padecimientos, estado=estados);

In [ ]:
mask=covid["PACIENTE"]=="HOSPITALIZADO"
mask1=covid["CONFIRMADO"]==True
mask2=covid["MUERTE x COVID"]==True
mask3=covid["ABREVIATURA"]=="MS"
covid[mask&mask1&mask3]["MUERTE x COVID"].mean()

In [ ]:
covid[mask&mask1]["MUERTE x COVID"].value_counts()

In [ ]:
covid[mask1]["PACIENTE"].value_counts()

In [ ]:
mask=covid["EDAD"].between(47,49)

mask1=covid["ABREVIATURA"]=="MS"
mask2=covid["H/M"]=="HOMBRE"
xx=covid[mask&mask1&mask2].sort_values("FECHA_INGRESO",ascending=False)[["FECHA_INGRESO","EDAD","H/M","MUNICIPIO","RESULTADOS","STATUS"]]

xx["STATUS"].value_counts(dropna=False,normalize=True)

In [ ]:
entidad.head()

In [ ]:
mask=entidad["ENTIDAD_FEDERATIVA"]=="MORELOS"
clave_estado=entidad[mask].iat[0,0]
inegi_estado=pd.DataFrame((pd.read_json("https://gaia.inegi.org.mx/wscatgeo/mgem/{}".format(clave_estado),
                                        orient='records',lines=True)).iloc[0,0])
inegi_estado["nom_agem"]=inegi_estado["nom_agem"].str.upper()
inegi_estado["ENT&MUN"]=inegi_estado["cve_agee"]+inegi_estado["cve_agem"]
inegi_estado.set_index("ENT&MUN",inplace=True)
inegi_estado.drop(["cve_agee","cve_agem","cve_cab","nom_cab"],inplace=True,axis=1)
inegi_estado.head()

In [ ]:
datos_edo=municipio[municipio["ENTIDAD_FEDERATIVA"]=="MORELOS"]
datos_edo.head()

In [ ]:
datos_edo=datos_edo.merge(inegi_estado["pob"],left_index=True, right_index=True).set_index("MUNICIPIO")
datos_edo.head()

In [ ]:
def resumen_por_municipio2(estado):

    mask0=entidad["ENTIDAD_FEDERATIVA"]==estado
    edo=entidad[mask0].iat[0,1]
    
    mask=covid["ABREVIATURA"]==edo
    mask2=municipio["ABREVIATURA"]==edo
    lista_munic=municipio["MUNICIPIO"][mask2].sort_values()
    mask3=covid["MUNICIPIO"].isin(lista_munic)
    munc=covid[mask3&mask]

    new=munc.pivot_table(values=["CONFIRMADO"],
                                 aggfunc=["sum"],index="FECHA_INGRESO",
                                 columns="MUNICIPIO")

    new.columns=new.columns.get_level_values(2)
    new=new.loc["2020-01-01":pd.Timestamp.today()-pd.Timedelta(5,unit="D")]
    new=new.fillna(0)

    cum=new.cumsum(skipna=True)
    rolling=new.rolling(21).mean().round(2)

    new_death=munc.pivot_table(values=["MUERTE x COVID"],
                                 aggfunc=["sum"],index="FECHA_INGRESO",
                                 columns="MUNICIPIO")
    new_death.columns=new_death.columns.get_level_values(2)
    new_death=new_death.loc["2020-01-01":pd.Timestamp.today()-pd.Timedelta(5,unit="D")]
    new_death=new_death.fillna(0)

    cum_death=new_death.cumsum()
    roll_death=new_death.rolling(21).mean().round(2)


    resumen=municipio["MUNICIPIO"][mask2].sort_values().to_frame().set_index("MUNICIPIO")
    resumen=resumen.drop("NO ESPECIFICADO")
    #resumen=resumen.merge(datos_edo["pob"],left_index=True, right_index=True)

    NUEVOS_CASOS=new.iloc[-1,:]
    NUEVOS_CASOS.name="NUEVOS CASOS"
    resumen=resumen.merge(NUEVOS_CASOS, left_index=True, right_index=True)

    PROMEDIO_NUEVOS_CASOS=rolling.iloc[-1,:].round(2)
    PROMEDIO_NUEVOS_CASOS.name="PROMEDIO NUEVOS CASOS"
    resumen=resumen.merge(PROMEDIO_NUEVOS_CASOS, left_index=True, right_index=True)

    CAMBIO_EN_PROMEDIO_DIARIO=(rolling.iloc[-1,:]-rolling.iloc[-5,:])/rolling.iloc[-5,:].round(2)
    CAMBIO_EN_PROMEDIO_DIARIO.name="CAMBIO EN PROMEDIO DIARIO"
    resumen=resumen.merge(CAMBIO_EN_PROMEDIO_DIARIO, left_index=True, right_index=True)

    CASOS_TOTALES=cum.iloc[-1,:]
    CASOS_TOTALES.name="CASOS TOTALES"
    resumen=resumen.merge(CASOS_TOTALES, left_index=True, right_index=True)

    ULTIMOS_DECESOS=new_death.iloc[-1,:]
    ULTIMOS_DECESOS.name="ULTIMOS DECESOS"
    resumen=resumen.merge(ULTIMOS_DECESOS, left_index=True, right_index=True)

    PROMEDIO_MUERTES=roll_death.iloc[-1,:].round(2)
    PROMEDIO_MUERTES.name="PROMEDIO MUERTES"
    resumen=resumen.merge(PROMEDIO_MUERTES, left_index=True, right_index=True)

    CAMBIO_EN_PROMEDIO_DE_DECESOS=(roll_death.iloc[-1,:]-roll_death.iloc[-5,:])/roll_death.iloc[-5,:].round(2)
    CAMBIO_EN_PROMEDIO_DE_DECESOS.name="CAMBIO EN PROMEDIO DE DECESOS"
    resumen=resumen.merge(CAMBIO_EN_PROMEDIO_DE_DECESOS, left_index=True, right_index=True)

    DECESOS_TOTALES=cum_death.iloc[-1,:]
    DECESOS_TOTALES.name="DECESOS TOTALES"
    resumen=resumen.merge(DECESOS_TOTALES, left_index=True, right_index=True)


    resumen["MORTALITY"]=resumen["DECESOS TOTALES"]/resumen["CASOS TOTALES"]
    resumen=resumen.replace([np.inf,-np.inf],0)

    resumen=resumen.style.format({'MORTALITY':"{:.2%}","CAMBIO EN PROMEDIO DIARIO":"{:.2%}"
                             ,"CAMBIO EN PROMEDIO DE DECESOS":"{:.2%}","NUEVOS CASOS":'{:,}',"CASOS TOTALES":'{:,}',
                           "PROMEDIO NUEVOS CASOS":'{:,}',"ULTIMOS DECESOS":'{:,}',"PROMEDIO MUERTES":'{:,}',
                           "DECESOS TOTALES":'{:,}',"POBLACION":'{:,}'},na_rep="-")

    from matplotlib import cm
    cm1=sns.diverging_palette(220, 22, sep=20, as_cmap=True)
    cm=sns.diverging_palette(220, 20, sep=20, as_cmap=True)
    resumen.background_gradient(cmap=cm1,axis=0,low=-0.1,high=0.3, 
                            subset=["MORTALITY","CAMBIO EN PROMEDIO DE DECESOS",
                                    "CAMBIO EN PROMEDIO DIARIO"])
    resumen.background_gradient(cmap=cm,axis=0,low=-0.3,high=0.3, 
                            subset=["CASOS TOTALES", "NUEVOS CASOS","DECESOS TOTALES"])
        
    return resumen

In [ ]:
resumen_por_municipio2("CDMX")

In [ ]:
pd.read_excel("Catalogos_0412.xlsx","Catálogo MUNICIPIOS", dtype='object')